In [1]:
from typing import List, Dict, Tuple, Iterator, Any
import requests
from bs4 import BeautifulSoup, Tag
import time
import threading
import concurrent.futures
import random
import time
from datetime import datetime
import re


In [2]:
import pandas as pd
import numpy as np

In [3]:
# Constants
BASE_URL: str = 'https://www.gsmarena.com'

HEADERS: Dict[str, str] = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 '
                  'Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,'
              'application/signed-exchange;v=b3;q=0.7',
    'Accept-Language': 'en-US,en;q=0.9'
}

## with proxy

In [4]:
url_proxy = 'https://www.us-proxy.org/'
ip_addresses = ['155.94.241.133' ,
'5.161.103.41:88' ,
'104.236.195.90:10009' ,
'172.173.132.85:80' ,
'93.188.161.84:80',
'68.183.48.146:10006',
'143.198.226.25:80',
'198.176.56.42:80',
'31.220.56.210:80',
'155.94.241.134:3128',
'138.68.235.51:80',
'202.5.16.44:80',
'155.94.241.132:3128',
'155.94.241.130:3128',
'72.169.67.61:87',
'34.122.187.196:80',
'198.176.56.39:80',
'104.45.128.122:80',
'216.137.184.253:80',
'198.176.56.43:80']

In [5]:


'''
def proxy_request(url, headers, ip_addresses):
    while True:
        try:
            proxy = random.randint(0, len(ip_addresses) - 1)
            print(f'which proxy?:{proxy}')
            proxies = {"http": ip_addresses[proxy]}
            response = requests.get(url, proxies=proxies, timeout=25, headers=headers)
            if response.status_code == 200:
                return response
            else:
                print(f"Failed to fetch page: {url}")
        except Exception as e:
            print(f"Error while making a request: {str(e)}")
            pass
'''
'''
def proxy_request(url, headers, ip_addresses):
    backoff_time = 2
    max_try = 3

    for _ in range(max_try):
        try:
            proxy = random.choice(ip_addresses)
            proxies = {"http": f"http://{proxy}"}
            response = requests.get(url, proxies=proxies, timeout=25, headers=HEADERS)
            response.raise_for_status()
            return response
        except requests.exceptions.HTTPError as e:
            if e.response.status_code == 429:
                print(f"Rate limited. Waiting for {backoff_time} seconds.")
                time.sleep(backoff_time)
                backoff_time *= 2
            else:
                raise
        except requests.RequestException as e:
            print(f"Error while making a request: {str(e)}")
            break

    return None
'''

import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

# Set up a requests session with retries
session = requests.Session()
retry_strategy = Retry(
    total=5,
    status_forcelist=[429],
    backoff_factor=1
)
adapter = HTTPAdapter(max_retries=retry_strategy)
session.mount('https://', adapter)
session.mount('http://', adapter)

def proxy_request(url: str, headers: Dict[str, str], proxies: Dict[str, str]) -> requests.Response:
    try:
        response = session.get(url, headers=headers, proxies=proxies, timeout=25)
        response.raise_for_status()
        return response
    except requests.exceptions.HTTPError as e:
        print(f"HTTP error occurred: {e}")
    except requests.exceptions.ConnectionError as e:
        print(f"Connection error occurred: {e}")
    except requests.exceptions.Timeout as e:
        print(f"Timeout error occurred: {e}")
    except requests.exceptions.RequestException as e:
        print(f"Error occurred: {e}")

    return None


In [6]:
# برند های مورد نیاز
def get_all_brands(content: BeautifulSoup) -> Dict[str, str]:
    brands: Dict[str, str] = {}
    for item in content.select('.brandmenu-v2.light.l-box.clearfix ul li'):
        brand_name = item.text.upper()
        if brand_name in ['ALCATEL', 'APPLE', 'ASUS', 'BLU', 'HTC', 'HUAWEI', 'INFINIX', 'LENOVO', 'LG', 'NOKIA', 'SONY', 'XIAOMI', 'ZTE', 'SAMSUNG']:
            brands[brand_name] = item.select_one('a')['href']
    return brands


#  مجموعه داده های مورد نیاز
def fetch_phone_data(phone_url: str) -> Dict[str, Any]:
    full_url = f"{BASE_URL}/{phone_url}"
    response = proxy_request(full_url, HEADERS, ip_addresses)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        name = soup.select_one('[data-spec="modelname"]').text if soup.select_one('[data-spec="modelname"]') else 'Unknown'
        # Simplified for demonstration, extend as needed
        return {'name': name, 'url': full_url}
    else:
        return {'error': 'Failed to fetch phone data because of response is not 200', 'url': full_url}



def phone_href(brand_url: str):
    phones_href = []
    next_page_url = f"{BASE_URL}/{brand_url}"
    while next_page_url:
        response = proxy_request(next_page_url, HEADERS, ip_addresses)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            phones = soup.select('.makers ul li a')
            year_before_2010_found = False
            for phone in phones:
                time.sleep(1)
                img_tag = phone.find('img')
                if img_tag and 'title' in img_tag.attrs:
                    title_text = img_tag['title']
                    year_announced = int(title_text.split('Announced ')[1].split('.')[0].split(' ')[-1])
                    if year_announced > 2010:
                        phones_href.append(phone['href'])
                    else:
                        year_before_2010_found = True
                        break
            if year_before_2010_found:
                print("Found a phone announced before 2010, stopping.")
                break

            next_page_link = soup.select_one('.nav-pages a.prevnextbutton[title="Next page"]')
            if next_page_link and 'href' in next_page_link.attrs:
                next_page_url = f"{BASE_URL}/{next_page_link['href']}"
            else:
                print('not more pages')
                next_page_url = None
        else:
            print(f"Failed to fetch page: {next_page_url}")
            break

    return phones_href



In [7]:
if __name__ == '__main__':
    # Randomly select a proxy to use for the request
    proxy_ip = random.choice(ip_addresses)
    proxies = {"http": f"http://{proxy_ip}", "https": f"http://{proxy_ip}"}

    # Attempt to make the request to the homepage
    home_page_response = proxy_request(BASE_URL, HEADERS, proxies)

    if home_page_response:
        soup = BeautifulSoup(home_page_response.content, 'html.parser')
        brands = get_all_brands(soup)
        print("Brands found:", brands)
        # Further logic goes here...
    else:
        print('Failed to fetch the homepage.')

Connection error occurred: HTTPSConnectionPool(host='www.gsmarena.com', port=443): Max retries exceeded with url: / (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x00000239B805DB50>, 'Connection to 104.45.128.122 timed out. (connect timeout=25)'))
Failed to fetch the homepage.


In [11]:
brand = 'APPLE'
path = brands['APPLE']
print(f"Fetching phones for {brand}...")
print(path)
phones = phone_href(path)
print(len(phones))
for phone in phones:
    print(phone)

Fetching phones for APPLE...
apple-phones-48.php
Error while making a request: 429 Client Error: Too Many Requests for url: https://www.gsmarena.com/apple-phones-48.php


AttributeError: 'NoneType' object has no attribute 'status_code'

## without proxy

In [31]:
def make_request(url:str, headers:Dict[str, str]) -> requests.Response:
    try:
        time.sleep(3)
        response = requests.get(url, timeout=25, headers=headers)
        response.raise_for_status()  # Raises a HTTPError if the response status code is 4XX/5XX
        return response
    except requests.RequestException as e:
        print(f"Error while making a request: {str(e)}")
        return None

In [32]:
# برند های مورد نیاز
def get_all_brands(content: BeautifulSoup) -> Dict[str, str]:
    brands: Dict[str, str] = {}
    for item in content.select('.brandmenu-v2.light.l-box.clearfix ul li'):
        brand_name = item.text.upper()
        if brand_name in ['ALCATEL', 'APPLE', 'ASUS', 'BLU', 'HTC', 'HUAWEI', 'INFINIX', 'LENOVO', 'LG', 'NOKIA', 'SONY', 'XIAOMI', 'ZTE', 'SAMSUNG']:
            brands[brand_name] = item.select_one('a')['href']
    return brands


#  مجموعه داده های مورد نیاز
def fetch_phone_data( phone_url:str ) -> Dict[str, Any]:
    full_url = f"{BASE_URL}/{phone_url}"
    response = make_request(full_url, HEADERS)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')

        # phone name
        name = soup.select_one('[data-spec="modelname"]').text if soup.select_one('[data-spec="modelname"]') else 'Unknown'

        # network
        network=soup.select_one('[data-spec="nettech"]').text if soup.select_one('[data-spec="nettech"]') else 'Unknown'

        #launch
        #announcedDate=datetime.strptime(soup.select_one('[data-spec="year"]').text,'%Y, %B %d') if soup.select_one('[data-spec="year"]') else 'Unknown'
        announced_text = soup.select_one('[data-spec="year"]').text if soup.select_one('[data-spec="year"]') else None
        if announced_text:
            match = re.search(r'(\d{4}), (\w+)', announced_text)
            if match:
                year, month = match.groups()
                try:
                    announcedDate = datetime.strptime(f'{year} {month}', '%Y %B').strftime('%Y-%m')
                except ValueError:
                    announcedDate = 'Unknown'
            else :
                announcedDate = 'Unknown'

        #releaseDate=datetime.strptime(soup.select_one('[data-spec="status"]').text.split("Released")[1][1:],'%Y, %B %d') if soup.select_one('[data-spec="status"]') else 'Unknown'
        releaseDate = 'Unknown'
        if status_text and "Released" in status_text:
            try:
                release_date_text = status_text.split("Released")[1].strip()
                releaseDate = datetime.strptime(release_date_text, '%Y, %B %d').strftime('%Y-%m')
            except ValueError:
                releaseDate = 'Unknown'
        availability=soup.select_one('[data-spec="status"]').text.split(".")[0] if soup.select_one('[data-spec="status"]') else 'Unknown'

        #body
        dimensions=soup.select_one('[data-spec="dimensions"]').text.split("mm")[0] if soup.select_one('[data-spec="dimensions"]') else 'Unknown'
        weight=float(soup.select_one('[data-spec="weight"]').text.split('g')[0]) if soup.select_one('[data-spec="weight"]') else 'Unknown'
        build=soup.select_one('[data-spec="build"]').text.split(",") if soup.select_one('[data-spec="build"]') else "unknown"
        sim=soup.select_one('[data-spec="sim"]').text.split(",") if soup.select_one('[data-spec="sim"]') else 'Unknown'

        #display
        displaytype=soup.select_one('[data-spec="displaytype"]').text.split(",")[0] if soup.select_one('[data-spec="displaytype"]') else 'Unknown'

        display_size_text = soup.select_one('[data-spec="displaysize"]').text if soup.select_one('[data-spec="displaysize"]') else ''
        display_parts = display_size_text.split(", ")
        displaysizeI = 'Unknown'
        displaysizeCM = 'Unknown'
        if len(display_parts) > 0:
            displaysizeI = float(display_parts[0].split(" inches")[0]) if "inches" in display_parts[0] else 'Unknown'
        if len(display_parts) > 1:
            displaysizeCM = float(display_parts[1].split(" cm")[0]) if "cm" in display_parts[1] else 'Unknown'
        #displaysizeI=float(soup.select_one('[data-spec="displaysize"]').text.split("inches")[0]) if soup.select_one('[data-spec="displaysize"]') else 'Unknown'
        #displaysizeCM=float(soup.select_one('[data-spec="displaysize"]').text.split(",")[1].split("cm")[0]) if soup.select_one('[data-spec="displaysize"]') else 'Unknown'

        resolution=soup.select_one('[data-spec="displayresolution"]').text.split("pixels")[0] if soup.select_one('[data-spec="displayresolution"]') else 'Unknown'
        protection=soup.select_one('[data-spec="displayprotection"]').text if soup.select_one('[data-spec="displayprotection"]') else "unknown"

        #os
        ostype=soup.select_one('[data-spec="os"]').text.split(",")[0].split()[0] if soup.select_one('[data-spec="os"]') else 'Unknown'
        osversion=soup.select_one('[data-spec="os"]').text.split(",")[0].split()[1] if soup.select_one('[data-spec="os"]') else 'Unknown'
        chipset=soup.select_one('[data-spec="chipset"]').text if soup.select_one('[data-spec="chipset"]') else 'Unknown'
        cpu=soup.select_one('[data-spec="cpu"]').text.split(" (")[0] if soup.select_one('[data-spec="cpu"]') else 'Unknown'
        gpu=soup.select_one('[data-spec="gpu"]').text.split(" (")[0] if soup.select_one('[data-spec="gpu"]') else "unknown"

        #memory
        cardslot=soup.select_one('[data-spec="memoryslot"]').text if soup.select_one('[data-spec="memoryslot"]') else 'Unknown'
        internal=soup.select_one('[data-spec="internalmemory"]').text.split(",") if soup.select_one('[data-spec="internalmemory"]') else 'Unknown'

        #main camera
        mainCam=soup.select_one('[data-spec="cam1modules"]').text if soup.select_one('[data-spec="cam1modules"]') else 'Unknown'
        mainCamNum=mainCam.count("MP")
        mainCamFeatures=soup.select_one('[data-spec="cam1features"]').text.split(",") if soup.select_one('[data-spec="cam1features"]') else 'Unknown'
        mainCamVideo=soup.select_one('[data-spec="cam1video"]').text.split(",") if soup.select_one('[data-spec="cam1video"]') else 'Unknown'

        #selfie cam
        selfieCam=soup.select_one('[data-spec="cam2modules"]').text if soup.select_one('[data-spec="cam2modules"]') else 'Unknown'
        selfieCamNum=selfieCam.count("MP")
        selfieCamFeatures=soup.select_one('[data-spec="cam2features"]').text.split(",") if soup.select_one('[data-spec="cam2features"]') else "unknown"
        selfieCamVideo=soup.select_one('[data-spec="cam2video"]').text.split(",") if soup.select_one('[data-spec="cam2video"]') else 'Unknown'

        #sound
        loudspeaker_info = soup.find('a', href="glossary.php3?term=loudspeaker").find_next_sibling('td')
        if loudspeaker_info:
            loudspeaker = loudspeaker_info.text.strip()
        else:
            loudspeaker = 'Unknown'
        ##Find the 3.5mm jack
        jack_info = soup.find('a', href="glossary.php3?term=audio-jack").find_next_sibling('td')
        if jack_info:
            jack_info = jack_info.text.strip()
        else:
            jack_info = 'Unknown'

        #comms
        wlan=soup.select_one('[data-spec="wlan"]').text.split(",") if soup.select_one('[data-spec="wlan"]') else 'Unknown'
        bluetooth=soup.select_one('[data-spec="bluetooth"]').text.split(",") if soup.select_one('[data-spec="bluetooth"]') else 'Unknown'
        gps=soup.select_one('[data-spec="gps"]').text.split(",") if soup.select_one('[data-spec="gps"]') else 'Unknown'
        nfc=soup.select_one('[data-spec="nfc"]').text if soup.select_one('[data-spec="nfc"]') else 'Unknown'
        radio=soup.select_one('[data-spec="radio"]').text if soup.select_one('[data-spec="radio"]') else 'Unknown'
        usb=soup.select_one('[data-spec="usb"]').text.split(",") if soup.select_one('[data-spec="usb"]') else 'Unknown'

        #features
        sensors=soup.select_one('[data-spec="sensors"]').text.split(",") if soup.select_one('[data-spec="sensors"]') else 'Unknown'

        #battery
        batteryType=soup.select_one('[data-spec="batdescription1"]').text if soup.select_one('[data-spec="batdescription1"]') else 'Unknown'
        charging_info_td = soup.find('td', text=lambda x: x and "Charging" in x).find_next_sibling('td') if soup.find('td', text=lambda x: x and "Charging" in x) else None
        charging = ' '.join(charging_info_td.stripped_strings) if charging_info_td else 'Unknown'

        #misc
        colors=soup.select_one('[data-spec="colors"]').text.split(",") if soup.select_one('[data-spec="colors"]') else 'Unknown'
        models=soup.select_one('[data-spec="models"]').text.split(",") if soup.select_one('[data-spec="models"]') else "unknown"

        #price
        price = soup.find('td', {'data-spec': 'price'}).text
        prices = {}
        for part in price.split('/'):
            part = part.strip()
            if '$' in part:
                prices['USD'] = ''.join(filter(lambda x: x.isdigit() or x == '.', part.replace("$", "").strip()) )
            elif '€' in part:
                prices['EUR'] = ''.join(filter(lambda x: x.isdigit() or x == '.', part.replace("€", "").strip()))
            elif '£' in part:
                prices['GBP'] = ''.join(filter(lambda x: x.isdigit() or x == '.', part.replace("£", "").strip()))
            elif '₹' in part:
                prices['INR'] = ''.join(filter(lambda x: x.isdigit() or x == '.', part.replace("₹", "").strip()))
        #######################################
        # save on  to dict
        phone_data={'url':full_url,"name":name,"network":network,"announcedDate":announcedDate,"releaseDate":releaseDate,
        "availability":availability,"dimensions":dimensions,"weight":weight,"build":build,"sim":sim,"displaytype":displaytype,
        "displaysizeI":displaysizeI,"displaysizeCM":displaysizeCM,"resolution":resolution,"protection":protection,
        "ostype":ostype,"osversion":osversion,"chipset":chipset,"cpu":cpu,"gpu":gpu,"cardslot":cardslot,"internal":internal,
        "mainCam":mainCam,"mainCamNum":mainCamNum,"mainCamFeatures":mainCamFeatures,"mainCamVideo":mainCamVideo,
        "selfieCam":selfieCam,"selfieCamNum":selfieCamNum,"selfieCamFeatures":selfieCamFeatures,"selfieCamVideo":selfieCamVideo,
        "loudspeaker":loudspeaker,"jack_info_3.5mm":jack_info,"wlan":wlan,"bluetooth":bluetooth,"gps":gps,
        "nfc":nfc,"radio":radio,"usb":usb,"sensors":sensors,"batteryType":batteryType,"charging":charging,
        "colors":colors,"models":models,"price":prices}
        #######################################
        return phone_data
    else:
        return {'error': 'Failed to fetch phone data because of response is not 200', 'url': full_url}


# آدرس url هر گوشی
def phone_href(brand_url: str):
    phones_href = []
    next_page_url = f"{BASE_URL}/{brand_url}"
    while next_page_url:
        response = make_request(next_page_url, HEADERS)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            phones = soup.select('.makers ul li a')
            year_before_2010_found = False
            for phone in phones:
                time.sleep(1)
                img_tag = phone.find('img')
                if img_tag and 'title' in img_tag.attrs:
                    title_text = img_tag['title']
                    year_announced = int(title_text.split('Announced ')[1].split('.')[0].split(' ')[-1])
                    if year_announced >= 2010:
                        phones_href.append(phone['href'])
                    else:
                        year_before_2010_found = True
                        break
            if year_before_2010_found:
                print("Found a phone announced before 2010, stopping.")
                break

            next_page_link = soup.select_one('.nav-pages a.prevnextbutton[title="Next page"]')
            if next_page_link and 'href' in next_page_link.attrs:
                next_page_url = f"{BASE_URL}/{next_page_link['href']}"
            else:
                print('not more pages')
                next_page_url = None
        else:
            print(f"Failed to fetch page: {next_page_url}")
            break

    return phones_href



In [28]:
status_text = 'Available. Released 2024, January 24'
releaseDate = 'Unknown'
if status_text and "Released" in status_text:
    try:
        release_date_text = status_text.lower().split("released")[1].strip()
        releaseDate = datetime.strptime(release_date_text, '%Y, %B %d').strftime('%Y-%m')
        print(releaseDate)
    except ValueError:
        releaseDate = 'Unknown'



2024-01


In [33]:
if __name__ == '__main__':
    home_page_response = make_request(BASE_URL, HEADERS)
    if home_page_response.status_code == 200:
        soup = BeautifulSoup(home_page_response.content, 'html.parser')
        brands = get_all_brands(soup)
        print(brands)
    else:
        print(f'the response is : {home_page_response.status_code}')


{'SAMSUNG': 'samsung-phones-9.php', 'APPLE': 'apple-phones-48.php', 'HUAWEI': 'huawei-phones-58.php', 'NOKIA': 'nokia-phones-1.php', 'SONY': 'sony-phones-7.php', 'LG': 'lg-phones-20.php', 'HTC': 'htc-phones-45.php', 'LENOVO': 'lenovo-phones-73.php', 'XIAOMI': 'xiaomi-phones-80.php', 'ASUS': 'asus-phones-46.php', 'ALCATEL': 'alcatel-phones-5.php', 'ZTE': 'zte-phones-62.php', 'INFINIX': 'infinix-phones-119.php'}


In [35]:
brand = 'APPLE'
path = brands['APPLE']
print(f"Fetching phones for {brand}...")
phones_href = phone_href(path)
print(len(phones_href))
for phone in phones_href:
    print(phone)

Fetching phones for APPLE...
Found a phone announced before 2010, stopping.
115
apple_iphone_15_pro_max-12548.php
apple_iphone_15_pro-12557.php
apple_iphone_15_plus-12558.php
apple_iphone_15-12559.php
apple_watch_ultra_2-12560.php
apple_watch_series_9-12561.php
apple_watch_series_9_aluminum-12562.php
apple_ipad_pro_12_9_(2022)-11939.php
apple_ipad_pro_11_(2022)-11940.php
apple_ipad_(2022)-11941.php
apple_iphone_14_pro_max-11773.php
apple_iphone_14_pro-11860.php
apple_iphone_14_plus-11862.php
apple_iphone_14-11861.php
apple_watch_ultra-11827.php
apple_watch_series_8-11866.php
apple_watch_series_8_aluminum-11864.php
apple_watch_se_(2022)-11865.php
apple_iphone_se_(2022)-11410.php
apple_ipad_air_(2022)-11411.php
apple_iphone_13_pro_max-11089.php
apple_iphone_13_pro-11102.php
apple_iphone_13-11103.php
apple_iphone_13_mini-11104.php
apple_ipad_mini_(2021)-11105.php
apple_ipad_10_2_(2021)-11106.php
apple_watch_edition_series_7-11147.php
apple_watch_series_7-11146.php
apple_watch_series_7_alu

In [36]:
#type(phones_href)
#print(phones_href[0])

all_phone_data_apple = []

for phone_url in phones_href:
    print(phone_url)
    phone_data = fetch_phone_data(phone_url)
    if 'error' not in phone_data:
        all_phone_data_apple.append(phone_data)
    else:
        print(f"Error fetching data for URL: {phone_url}")


apple_iphone_15_pro_max-12548.php


<ipython-input-32-b7d2323312e2>:120: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  charging_info_td = soup.find('td', text=lambda x: x and "Charging" in x).find_next_sibling('td') if soup.find('td', text=lambda x: x and "Charging" in x) else None


apple_iphone_15_pro-12557.php
apple_iphone_15_plus-12558.php
apple_iphone_15-12559.php
apple_watch_ultra_2-12560.php
apple_watch_series_9-12561.php
apple_watch_series_9_aluminum-12562.php
apple_ipad_pro_12_9_(2022)-11939.php
apple_ipad_pro_11_(2022)-11940.php
apple_ipad_(2022)-11941.php
apple_iphone_14_pro_max-11773.php
apple_iphone_14_pro-11860.php
apple_iphone_14_plus-11862.php
apple_iphone_14-11861.php
apple_watch_ultra-11827.php
apple_watch_series_8-11866.php
apple_watch_series_8_aluminum-11864.php
apple_watch_se_(2022)-11865.php
apple_iphone_se_(2022)-11410.php
apple_ipad_air_(2022)-11411.php
apple_iphone_13_pro_max-11089.php
apple_iphone_13_pro-11102.php
apple_iphone_13-11103.php
apple_iphone_13_mini-11104.php
apple_ipad_mini_(2021)-11105.php
apple_ipad_10_2_(2021)-11106.php
apple_watch_edition_series_7-11147.php
apple_watch_series_7-11146.php
apple_watch_series_7_aluminum-11107.php
apple_ipad_pro_12_9_(2021)-10864.php
apple_ipad_pro_11_(2021)-10865.php
apple_iphone_12_pro_max-10

In [39]:
phones_df_apple = pd.DataFrame(all_phone_data_apple)


In [40]:
phones_df_apple.to_excel('APPLE.xlsx')